<a href="https://colab.research.google.com/github/Stacy067/Deep-learning-and-Tensorflow/blob/main/TF_Certificate_Category_5_(%EC%8B%A4%EC%8A%B5)%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Category 5

Sequence (시계열) 데이터 다루기

## 확인

1. GPU 옵션 켜져 있는지 확인할 것!!! (수정 - 노트설정 - 하드웨어설정 (GPU))

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

## 문제

For this task you will need to train a neural network
to predict sunspot activity using the Sunspots.csv
provided. 

Your neural network is expected to have an MAE
of at least 20, with top marks going to one with an MAE
of around 15. 

At the bottom is provided some testing
code should you want to check before uploading which measures
the MAE for you. 

Strongly recommend you test your model with
this to be able to see how it performs.



-------------------------------
**Sequence(시퀀스)**

Sunspots.csv를 사용하여 **태양 흑점 활동(sunspot)을 예측하는 인공신경망을 만듭니다.

MAE 오차 기준으로 최소 20이하로 예측할 것을 권장하며, 탑 랭킹에 들려면 MAE 15 근처에 도달해야합니다.

아래 주어진 샘플코드는 당신의 모델을 테스트 하는 용도로 활용할 수 있습니다.

-----------------------------------


## STEP 1. 필요한 모듈 import

**[코드]**

In [4]:
import csv
import tensorflow as tf
import numpy as np
import urllib

# 이곳에 코드를 입력해 주세요
from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

## STEP 2. 데이터셋 다운로드

서버로부터 샘플 데이터셋을 다운로드 받습니다.

In [5]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

('sunspots.csv', <http.client.HTTPMessage at 0x7f1746eafad0>)

## STEP 3. 다운로드 받은 csv 파일로부터 데이터셋 만들기

csv.reader() 함수를 활용합니다.

* 첫번째 파라미터에는 file을 , delimiter에는 구분자를 넣어 줍니다.

빈 list를 만들어 줍니다. (sunspots, time_step)

**[코드]**

In [6]:
sunspots = []
time_step = []

`time_step`에는 **index** 값을, `sunspots`에는 sunspots의 정보를 넣어 줍니다.

**[코드]**

In [8]:
# 이곳에 코드를 입력해 주세요
with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

sunspots, time_step 확인

In [9]:
sunspots[:5]

[96.7, 104.3, 116.7, 92.8, 141.7]

In [10]:
time_step[:5]

[0, 1, 2, 3, 4]

sunspots와 time_step을 `numpy array`로 변환합니다.

* 참고: 모델은 list 타입을 받아들이지 못합니다. 따라서, numpy array 로 변환해 줍니다.

**[코드]**

In [12]:
series = np.array(sunspots)
time = np.array(time_step)

잘 분할되었는지 shape를 확인합니다.

In [13]:
series.shape, time.shape

((3235,), (3235,))

## STEP 4. Train Set, Validation Set 생성

3000 인덱스를 기준으로 Train / Validation Set를 분할 합니다.

In [14]:
split_time = 3000

**[코드]**

In [15]:
time_train = time[:split_time]
time_valid = time[split_time:]

x_train = series[:split_time]
x_valid = series[split_time:]

## STEP 5. Window Dataset Loader 생성

자세한 Dataset 활용법은 [블로그 링크](https://teddylee777.github.io/tensorflow/dataset-batch-window)를 참고해 보시고, 연습해보세요!

In [16]:
# 윈도우 사이즈
window_size=30
# 배치 사이즈
batch_size = 32
# 셔플 사이즈
shuffle_size = 1000

In [17]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

`train_set`와 `validation_set` 각각에 대한 **windowed_dataset**을 만듭니다.

**[코드]**

In [18]:
train_set = windowed_dataset(x_train,
                             window_size=window_size,
                             batch_size=batch_size,
                             shuffle_buffer=shuffle_size)


validation_set = windowed_dataset(x_valid,
                                  window_size=window_size,
                                  batch_size=batch_size,
                                  shuffle_buffer=shuffle_size)

## STEP 6. 모델 정의 (Sequential)

**[코드]**

In [19]:
model = Sequential([
    tf.keras.layers.Conv1D(60, kernel_size=5,
                           padding="causal",
                           activation="relu",
                           input_shape=[None, 1]),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)
])

모델의 구조 요약을 확인합니다.

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 60)          360       
                                                                 
 lstm (LSTM)                 (None, None, 60)          29040     
                                                                 
 lstm_1 (LSTM)               (None, None, 60)          29040     
                                                                 
 dense (Dense)               (None, None, 30)          1830      
                                                                 
 dense_1 (Dense)             (None, None, 10)          310       
                                                                 
 dense_2 (Dense)             (None, None, 1)           11        
                                                                 
 lambda (Lambda)             (None, None, 1)           0

## STEP 7. 컴파일 (compile)

**Optimizer**는 SGD(Stochastic Gradient Descent) 를 사용합니다.

* lr(learning_rate): 학습률입니다.
* momentum: 모멘텀 (가중치) 입니다.

**[코드]**

In [21]:
optimizer = SGD(learning_rate=1e-5, momentum=0.9)

**Huber Loss**: MSE와 MAE를 절충한 후버 손실(Huber loss)

\begin{split}L_{\delta}=\left\{\begin{matrix}
\frac{1}{2}(y - \hat{y})^{2} & if \left | (y - \hat{y})  \right | < \delta\\
\delta ((y - \hat{y}) - \frac1 2 \delta) & otherwise
\end{matrix}\right.\end{split}

**[코드]**

In [22]:
loss = Huber()

model.compile()시 우리가 튜닝한 **optimizer**와 **loss**를 활용합니다.

**[코드]**

In [23]:
model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['mae'])

## STEP 8. ModelCheckpoint: 체크포인트 생성

`val_loss` 기준으로 epoch 마다 최적의 모델을 저장하기 위하여, ModelCheckpoint를 만듭니다.
* `checkpoint_path`는 모델이 저장될 파일 명을 설정합니다.
* `ModelCheckpoint`을 선언하고, 적절한 옵션 값을 지정합니다.

**[코드]**

In [24]:
checkpoint_path = "tmp_checkpoint.ckpt"
checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             monitor='val_mae',
                             verbose=1)

## STEP 9. 학습 (fit)

In [25]:
epochs=100

**[코드]**

In [26]:
model.fit(train_set,
          validation_data=(validation_set),
          epochs=epochs,
          callbacks=[checkpoint],
          )

Epoch 1/100
     93/Unknown - 19s 13ms/step - loss: 26.5750 - mae: 27.0696
Epoch 1: saving model to tmp_checkpoint.ckpt
93/93 [==============================] - 20s 26ms/step - loss: 26.5750 - mae: 27.0696 - val_loss: 17.7742 - val_mae: 18.2666
Epoch 2/100
91/93 [============================>.] - ETA: 0s - loss: 20.7550 - mae: 21.2486
Epoch 2: saving model to tmp_checkpoint.ckpt
93/93 [==============================] - 1s 14ms/step - loss: 20.7882 - mae: 21.2818 - val_loss: 15.1845 - val_mae: 15.6768
Epoch 3/100
92/93 [============================>.] - ETA: 0s - loss: 20.0346 - mae: 20.5282
Epoch 3: saving model to tmp_checkpoint.ckpt
93/93 [==============================] - 1s 14ms/step - loss: 20.0324 - mae: 20.5260 - val_loss: 15.0039 - val_mae: 15.4943
Epoch 4/100
88/93 [===========================>..] - ETA: 0s - loss: 19.2303 - mae: 19.7231
Epoch 4: saving model to tmp_checkpoint.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 19.2932 - mae: 19.7859 - val_loss:

## STEP 10. 학습 완료 후 Load Weights (ModelCheckpoint)

학습이 완료된 후에는 반드시 `load_weights`를 해주어야 합니다.

그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.

**[코드]**

In [27]:
model.load_weights(checkpoint_path)